In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold

In [2]:
train=pd.read_csv('/kaggle/input/playground-series-s6e1/train.csv')
test=pd.read_csv('/kaggle/input/playground-series-s6e1/test.csv')

In [3]:
train.head(2)

,id,age,gender,course,study_hours,class_attendance,internet_access,sleep_hours,sleep_quality,study_method,facility_rating,exam_difficulty,exam_score
0,0,21,female,b.sc,7.91,98.8,no,4.9,average,online videos,low,easy,78.3
1,1,18,other,diploma,4.95,94.8,yes,4.7,poor,self-study,medium,moderate,46.7


In [4]:
train.columns

Index(['id', 'age', 'gender', 'course', 'study_hours', 'class_attendance',
       'internet_access', 'sleep_hours', 'sleep_quality', 'study_method',
       'facility_rating', 'exam_difficulty', 'exam_score'],
      dtype='object')

In [5]:
train.shape

(630000, 13)

In [6]:
train.isna().sum()

id                  0
age                 0
gender              0
course              0
study_hours         0
class_attendance    0
internet_access     0
sleep_hours         0
sleep_quality       0
study_method        0
facility_rating     0
exam_difficulty     0
exam_score          0
dtype: int64

In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 630000 entries, 0 to 629999
Data columns (total 13 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   id                630000 non-null  int64  
 1   age               630000 non-null  int64  
 2   gender            630000 non-null  object 
 3   course            630000 non-null  object 
 4   study_hours       630000 non-null  float64
 5   class_attendance  630000 non-null  float64
 6   internet_access   630000 non-null  object 
 7   sleep_hours       630000 non-null  float64
 8   sleep_quality     630000 non-null  object 
 9   study_method      630000 non-null  object 
 10  facility_rating   630000 non-null  object 
 11  exam_difficulty   630000 non-null  object 
 12  exam_score        630000 non-null  float64
dtypes: float64(4), int64(2), object(7)
memory usage: 62.5+ MB


In [8]:
train.describe()

,id,age,study_hours,class_attendance,sleep_hours,exam_score
count,630000.000000,630000.000000,630000.000000,630000.000000,630000.000000,630000.000000
mean,314999.500000,20.545821,4.002337,71.987261,7.072758,62.506672
std,181865.479132,2.260238,2.359880,17.430098,1.744811,18.916884
min,0.000000,17.000000,0.080000,40.600000,4.100000,19.599000
25%,157499.750000,19.000000,1.970000,57.000000,5.600000,48.800000
50%,314999.500000,21.000000,4.000000,72.600000,7.100000,62.600000
75%,472499.250000,23.000000,6.050000,87.200000,8.600000,76.300000
max,629999.000000,24.000000,7.910000,99.400000,9.900000,100.000000


In [9]:
test.head(3)

,id,age,gender,course,study_hours,class_attendance,internet_access,sleep_hours,sleep_quality,study_method,facility_rating,exam_difficulty
0,630000,24,other,ba,6.85,65.2,yes,5.2,poor,group study,high,easy
1,630001,18,male,diploma,6.61,45.0,no,9.3,poor,coaching,low,easy
2,630002,24,female,b.tech,6.60,98.5,yes,6.2,good,group study,medium,moderate


In [10]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 270000 entries, 0 to 269999
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   id                270000 non-null  int64  
 1   age               270000 non-null  int64  
 2   gender            270000 non-null  object 
 3   course            270000 non-null  object 
 4   study_hours       270000 non-null  float64
 5   class_attendance  270000 non-null  float64
 6   internet_access   270000 non-null  object 
 7   sleep_hours       270000 non-null  float64
 8   sleep_quality     270000 non-null  object 
 9   study_method      270000 non-null  object 
 10  facility_rating   270000 non-null  object 
 11  exam_difficulty   270000 non-null  object 
dtypes: float64(3), int64(2), object(7)
memory usage: 24.7+ MB


In [11]:
test.describe()

,id,age,study_hours,class_attendance,sleep_hours
count,270000.000000,270000.000000,270000.000000,270000.000000,270000.000000
mean,764999.500000,20.544137,4.003878,71.982509,7.072070
std,77942.430678,2.260452,2.357741,17.414695,1.745513
min,630000.000000,17.000000,0.080000,40.600000,4.100000
25%,697499.750000,19.000000,1.980000,57.000000,5.600000
50%,764999.500000,21.000000,4.010000,72.600000,7.100000
75%,832499.250000,23.000000,6.050000,87.200000,8.600000
max,899999.000000,24.000000,7.910000,99.400000,9.900000


In [12]:
test.isna().sum()

id                  0
age                 0
gender              0
course              0
study_hours         0
class_attendance    0
internet_access     0
sleep_hours         0
sleep_quality       0
study_method        0
facility_rating     0
exam_difficulty     0
dtype: int64

In [13]:
X=train.drop(["id","exam_score"],axis=1)
test_id=test['id']
target=train['exam_score']
test=test.drop(['id'],axis=1)

In [14]:
#cat_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
#num_cols = X.select_dtypes(include=['number']).columns.tolist()

In [15]:
cat_cols = X.select_dtypes(include=['object']).columns.tolist()

for col in cat_cols:
    X[col] = X[col].astype('category')
    test[col] = test[col].astype('category')

In [16]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

/usr/local/lib/python3.12/dist-packages/sqlalchemy/orm/query.py:195: SyntaxWarning: "is not" with 'tuple' literal. Did you mean "!="?
  if entities is not ():


In [17]:
X_train,X_test,y_train,y_test=train_test_split(X,target,test_size=0.2,random_state=42)

In [18]:
"""""import optuna
import lightgbm as lgb
import numpy as np
import gc
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# --- STEP 1: Split once outside the loop to save memory ---
X_train_opt, X_val_opt, y_train_opt, y_val_opt = train_test_split(
    X, target, test_size=0.2, random_state=42
)

def objective(trial):
    # 1. Widened Parameter Search Space
    params = {
        "objective": "regression",
        "metric": "rmse",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "random_state": 42,
        "device": "gpu",
        "gpu_platform_id": 0,
        "gpu_device_id": 0,
        
        # --- ARCHITECTURE ---
        # Learning rate: Lower floor allows for more precise convergence
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.2, log=True),
        # n_estimators: Widened range; early stopping will handle the actual cutoff
        "n_estimators": trial.suggest_int("n_estimators", 1000, 5000),
        # num_leaves: The most important LGBM param. Widened significantly.
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        # max_depth: Widened, but num_leaves is the primary control
        "max_depth": trial.suggest_int("max_depth", 3, 15),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),

        # --- REGULARIZATION (The "Widen" Kit) ---
        # L1 and L2 regularization help prevent overfitting with high num_leaves
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True),
        # min_split_gain: Minimum loss reduction required to make a further partition
        "min_split_gain": trial.suggest_float("min_split_gain", 1e-8, 1.0, log=True),

        # --- SAMPLING ---
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "subsample_freq": trial.suggest_int("subsample_freq", 1, 7),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.4, 1.0),
    }

    # 2. Initialize Model
    model = lgb.LGBMRegressor(**params)
    
    # 3. Fit with Early Stopping
    try:
        model.fit(
            X_train_opt, y_train_opt,
            eval_set=[(X_val_opt, y_val_opt)],
            callbacks=[
                lgb.early_stopping(stopping_rounds=100),
                lgb.log_evaluation(period=0) # Keeps output clean
            ]
        )

        preds = model.predict(X_val_opt)
        rmse = np.sqrt(mean_squared_error(y_val_opt, preds))
    
    except Exception as e:
        print(f"Trial failed due to: {e}")
        return 1e9 # Return high error if GPU OOMs
    
    finally:
        # 4. CRITICAL: Clean up memory after every trial
        if 'model' in locals(): del model
        gc.collect()

    return rmse

# 5. Run the Study
# Using more trials because the search space is now much larger
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=20) 

print("\n" + "="*30)
print("Best Parameters found:", study.best_params)"""""

'""import optuna\nimport lightgbm as lgb\nimport numpy as np\nimport gc\nfrom sklearn.model_selection import train_test_split\nfrom sklearn.metrics import mean_squared_error\n\n# --- STEP 1: Split once outside the loop to save memory ---\nX_train_opt, X_val_opt, y_train_opt, y_val_opt = train_test_split(\n    X, target, test_size=0.2, random_state=42\n)\n\ndef objective(trial):\n    # 1. Widened Parameter Search Space\n    params = {\n        "objective": "regression",\n        "metric": "rmse",\n        "verbosity": -1,\n        "boosting_type": "gbdt",\n        "random_state": 42,\n        "device": "gpu",\n        "gpu_platform_id": 0,\n        "gpu_device_id": 0,\n        \n        # --- ARCHITECTURE ---\n        # Learning rate: Lower floor allows for more precise convergence\n        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.2, log=True),\n        # n_estimators: Widened range; early stopping will handle the actual cutoff\n        "n_estimators": trial.sugges

In [19]:
best_params= {
    'learning_rate': 0.03211241359721744, 
    'n_estimators': 1189, 
    'num_leaves': 110, 
    'max_depth': 7, 
    'min_child_samples': 59, 
    'reg_alpha': 0.09877798108894997, 
    'reg_lambda': 0.5104452770153255,
    'min_split_gain': 5.4230114411749396e-05, 
    'subsample': 0.9795690904305359, 
    'subsample_freq': 2, 
    'colsample_bytree': 0.675889588384865,
    "device": "gpu",            
    "gpu_platform_id": 0,       
    "gpu_device_id": 0,
    "verbosity": -1,
    "random_state": 42
}

In [20]:
"""""model.fit(
    X_train, y_train,
    eval_set=[(X_test, y_test)],
    callbacks=[
        lgb.early_stopping(stopping_rounds=50),
        lgb.log_evaluation(period=100)
    ]
)"""""

'""model.fit(\n    X_train, y_train,\n    eval_set=[(X_test, y_test)],\n    callbacks=[\n        lgb.early_stopping(stopping_rounds=50),\n        lgb.log_evaluation(period=100)\n    ]\n)'

In [21]:
n_splits = 10
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

oof_preds = np.zeros(len(X))
test_preds = np.zeros(len(test))
rmse_scores = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X, target)):
    print(f"\n--- Training Fold {fold + 1} ---")
    
    # 1. Properly split the data using the fold indices
    X_train_fold, X_val_fold = X.iloc[train_idx], X.iloc[val_idx]
    y_train_fold, y_val_fold = target.iloc[train_idx], target.iloc[val_idx]
    
    # 2. Initialize model
    model = lgb.LGBMRegressor(
        **best_params,
        categorical_feature=cat_cols # Explicitly tell LGBM which are categorical
    )
    
    # 3. Fit on the FOLD data, not the global data
    model.fit(
        X_train_fold, y_train_fold,
        eval_set=[(X_val_fold, y_val_fold)],
        callbacks=[
            lgb.early_stopping(stopping_rounds=50),
            lgb.log_evaluation(period=100)
        ]
    )
    
    # 4. Predict on validation set
    val_preds = model.predict(X_val_fold)
    oof_preds[val_idx] = val_preds
    
    # 5. Calculate fold RMSE
    fold_rmse = np.sqrt(mean_squared_error(y_val_fold, val_preds))
    rmse_scores.append(fold_rmse)
    print(f"Fold {fold + 1} RMSE: {fold_rmse:.5f}")
    
    # 6. Accumulate test predictions (Averaging)
    test_preds += model.predict(test) / n_splits

# 7. Final Evaluation
total_rmse = np.sqrt(mean_squared_error(target, oof_preds))
print(f"\nOverall OOF RMSE: {total_rmse:.5f}")
print(f"Mean Fold RMSE: {np.mean(rmse_scores):.5f} (+/- {np.std(rmse_scores):.5f})")


--- Training Fold 1 ---


/usr/local/lib/python3.12/dist-packages/lightgbm/basic.py:2137: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(
/usr/local/lib/python3.12/dist-packages/lightgbm/basic.py:2159: UserWarning: categorical_feature in param dict is overridden.
  _log_warning(f"{cat_alias} in param dict is overridden.")
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warnin

Training until validation scores don't improve for 50 rounds
[100]	valid_0's l2: 80.6513
[200]	valid_0's l2: 76.6568
[300]	valid_0's l2: 76.3697
[400]	valid_0's l2: 76.2428
[500]	valid_0's l2: 76.1441
[600]	valid_0's l2: 76.0764
[700]	valid_0's l2: 76.0075
[800]	valid_0's l2: 75.9687
[900]	valid_0's l2: 75.9503
[1000]	valid_0's l2: 75.9236
[1100]	valid_0's l2: 75.9057
Did not meet early stopping. Best iteration is:
[1172]	valid_0's l2: 75.8978
Fold 1 RMSE: 8.71194

--- Training Fold 2 ---


/usr/local/lib/python3.12/dist-packages/lightgbm/basic.py:2137: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(
/usr/local/lib/python3.12/dist-packages/lightgbm/basic.py:2159: UserWarning: categorical_feature in param dict is overridden.
  _log_warning(f"{cat_alias} in param dict is overridden.")
/usr/local/lib/python3.12/dist-packages/lightgbm/basic.py:2137: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(
/usr/local/lib/python3.12/dist-packages/lightgbm/basic.py:2159: UserWarning: categorical_feature in param dict is overridden.
  _log_warning(f"{cat_alias} in param dict is overridden.")


Training until validation scores don't improve for 50 rounds
[100]	valid_0's l2: 81.4829
[200]	valid_0's l2: 77.5401
[300]	valid_0's l2: 77.2636
[400]	valid_0's l2: 77.1412
[500]	valid_0's l2: 77.0484
[600]	valid_0's l2: 76.9629
[700]	valid_0's l2: 76.893
[800]	valid_0's l2: 76.8485
[900]	valid_0's l2: 76.8208
[1000]	valid_0's l2: 76.7999
[1100]	valid_0's l2: 76.7811
Did not meet early stopping. Best iteration is:
[1173]	valid_0's l2: 76.7727
Fold 2 RMSE: 8.76201

--- Training Fold 3 ---


...

Training until validation scores don't improve for 50 rounds
[100]	valid_0's l2: 81.2751
[200]	valid_0's l2: 77.1319
[300]	valid_0's l2: 76.8032
[400]	valid_0's l2: 76.671
[500]	valid_0's l2: 76.5572
[600]	valid_0's l2: 76.4637
[700]	valid_0's l2: 76.3893
[800]	valid_0's l2: 76.3453
[900]	valid_0's l2: 76.3123
[1000]	valid_0's l2: 76.2771
[1100]	valid_0's l2: 76.2603
Did not meet early stopping. Best iteration is:
[1172]	valid_0's l2: 76.2505
Fold 8 RMSE: 8.73215

--- Training Fold 9 ---


/usr/local/lib/python3.12/dist-packages/lightgbm/basic.py:2137: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(
/usr/local/lib/python3.12/dist-packages/lightgbm/basic.py:2159: UserWarning: categorical_feature in param dict is overridden.
  _log_warning(f"{cat_alias} in param dict is overridden.")
/usr/local/lib/python3.12/dist-packages/lightgbm/basic.py:2137: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(
/usr/local/lib/python3.12/dist-packages/lightgbm/basic.py:2159: UserWarning: categorical_feature in param dict is overridden.
  _log_warning(f"{cat_alias} in param dict is overridden.")


Training until validation scores don't improve for 50 rounds
[100]	valid_0's l2: 81.6165
[200]	valid_0's l2: 77.5153
[300]	valid_0's l2: 77.199
[400]	valid_0's l2: 77.0681
[500]	valid_0's l2: 76.9657
[600]	valid_0's l2: 76.8916
[700]	valid_0's l2: 76.8285
[800]	valid_0's l2: 76.7881
[900]	valid_0's l2: 76.753
[1000]	valid_0's l2: 76.7215
[1100]	valid_0's l2: 76.7001
Did not meet early stopping. Best iteration is:
[1177]	valid_0's l2: 76.6828
Fold 9 RMSE: 8.75687

--- Training Fold 10 ---


/usr/local/lib/python3.12/dist-packages/lightgbm/basic.py:2137: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(
/usr/local/lib/python3.12/dist-packages/lightgbm/basic.py:2159: UserWarning: categorical_feature in param dict is overridden.
  _log_warning(f"{cat_alias} in param dict is overridden.")
/usr/local/lib/python3.12/dist-packages/lightgbm/basic.py:2137: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(
/usr/local/lib/python3.12/dist-packages/lightgbm/basic.py:2159: UserWarning: categorical_feature in param dict is overridden.
  _log_warning(f"{cat_alias} in param dict is overridden.")


Training until validation scores don't improve for 50 rounds
[100]	valid_0's l2: 82.1482
[200]	valid_0's l2: 77.9163
[300]	valid_0's l2: 77.5688
[400]	valid_0's l2: 77.4182
[500]	valid_0's l2: 77.3146
[600]	valid_0's l2: 77.2309
[700]	valid_0's l2: 77.1677
[800]	valid_0's l2: 77.1343
[900]	valid_0's l2: 77.0872
[1000]	valid_0's l2: 77.0618
[1100]	valid_0's l2: 77.0422
Did not meet early stopping. Best iteration is:
[1186]	valid_0's l2: 77.0317
Fold 10 RMSE: 8.77677

Overall OOF RMSE: 8.74481
Mean Fold RMSE: 8.74478 (+/- 0.02428)


In [22]:
submission=pd.DataFrame({
    "id":test_id,
    "exam_score":test_preds
})
submission.to_csv('submission.csv', index=False)
print("Submission file 'submission.csv' successfully created!")

Submission file 'submission.csv' successfully created!


In [23]:
submission.head(3)

,id,exam_score
0,630000,71.608119
1,630001,70.858820
2,630002,88.144486
